In [6]:
import pandas as pd
import numpy as np

def perform_feature_engineering_v2(input_path, output_path):
    """
    STEP 1 (CORRECTED): FEATURE ENGINEERING
    
    Fixes the issue where categorical columns were mistaken for numeric, causing 100% data loss.
    
    Goals:
    1. Robustly parse 'half' split times to seconds.
    2. Calculate Pacing Metrics (Pacing Index, % Slowdown).
    3. Create Performance Categories.
    4. Define 'The Wall'.
    
    Args:
        input_path (str): Path to the cleaned parquet file.
        output_path (str): Path to save the engineered dataset.
    """
    
    print(f"--- Starting Feature Engineering (v2) on {input_path} ---")
    
    # 1. LOAD DATA
    # ---------------------------------------------------------
    try:
        df = pd.read_parquet(input_path)
        print(f"Data Loaded successfully. Shape: {df.shape}")
    except Exception as e:
        print(f"Critical Error loading file: {e}")
        return None

    # 2. ROBUST TIME PARSING (THE FIX)
    # ---------------------------------------------------------
    
    def safe_parse_time(time_val):
        """
        Robustly converts various time formats to total seconds.
        Handles: Strings ('01:30:00'), Categories, Floats, and NaNs.
        """
        # Force conversion to string to handle 'category' dtype safely
        s = str(time_val).strip()
        
        # Check for empty/missing values common in CSVs
        if s.lower() in ['nan', 'none', '', '-', 'nat']:
            return np.nan
            
        try:
            parts = s.split(':')
            if len(parts) == 3: # Format H:M:S (e.g., 01:30:45)
                return int(parts[0])*3600 + int(parts[1])*60 + float(parts[2])
            elif len(parts) == 2: # Format M:S (e.g., 59:30)
                return int(parts[0])*60 + float(parts[1])
            else:
                # If it's just a number (e.g. '5400'), try returning it as float
                try:
                    return float(s)
                except:
                    return np.nan
        except:
            return np.nan

    print("Diagnosing 'half' column...")
    print(f"Original Dtype: {df['half'].dtype}")
    print(f"Sample values: {df['half'].head(3).tolist()}")

    # Apply the parsing logic explicitly
    print("Parsing 'half' split to seconds...")
    df['half_seconds'] = df['half'].apply(safe_parse_time)
    
    # Ensure 'time_seconds' (finish time) is also clean
    # (It should be from the previous step, but we double-check)
    df['time_seconds'] = pd.to_numeric(df['time_seconds'], errors='coerce')

    # 3. FILTERING VALID DATA
    # ---------------------------------------------------------
    initial_count = len(df)
    
    # We need both valid total time AND valid half split to calculate pacing
    df_engineered = df.dropna(subset=['time_seconds', 'half_seconds']).copy()
    
    # Sanity Check: Remove half marathons < 30 mins (1800s) -> Impossible/Error
    df_engineered = df_engineered[df_engineered['half_seconds'] > 1800]
    
    dropped_count = initial_count - len(df_engineered)
    print(f"Filtered {dropped_count} rows ({dropped_count/initial_count:.2%}) due to missing/invalid split data.")
    
    if len(df_engineered) == 0:
        print("CRITICAL ERROR: All rows were dropped. Check the 'Sample values' printed above.")
        return None

    # 4. CALCULATE PACING METRICS
    # ---------------------------------------------------------
    print("Calculating Pacing Index and Slowdown metrics...")
    
    # Second Half = Total Time - First Half
    df_engineered['second_half_seconds'] = df_engineered['time_seconds'] - df_engineered['half_seconds']
    
    # Pacing Index (PI)
    # PI = 1.0 (Even Split), PI > 1.0 (Positive Split/Slowdown)
    df_engineered['pacing_index'] = df_engineered['second_half_seconds'] / df_engineered['half_seconds']
    
    # % Slowdown 
    df_engineered['pct_slowdown'] = (df_engineered['pacing_index'] - 1) * 100

    # Define "Hitting The Wall" (Categorical)
    # Threshold: > 20% slowdown
    df_engineered['hit_the_wall'] = df_engineered['pct_slowdown'] > 20.0

    # 5. CREATE PERFORMANCE CATEGORIES
    # ---------------------------------------------------------
    print("Assigning Performance Categories...")
    
    conditions = [
        (df_engineered['time_seconds'] < 10800), # Sub 3h
        (df_engineered['time_seconds'] >= 10800) & (df_engineered['time_seconds'] < 12600), # 3h-3h30
        (df_engineered['time_seconds'] >= 12600) & (df_engineered['time_seconds'] < 14400), # 3h30-4h
        (df_engineered['time_seconds'] >= 14400) & (df_engineered['time_seconds'] < 16200), # 4h-4h30
        (df_engineered['time_seconds'] >= 16200) # > 4h30
    ]
    
    category_labels = [
        '1. Elite/Sub-3h (<3:00)',
        '2. Advanced (3:00-3:30)',
        '3. Intermediate (3:30-4:00)',
        '4. Recreational (4:00-4:30)',
        '5. Casual (>4:30)'
    ]
    
    df_engineered['performance_level'] = np.select(conditions, category_labels, default='Unknown')

    # 6. FINAL CLEANUP & SAVING
    # ---------------------------------------------------------
    # Remove extreme pacing outliers (PI < 0.5 or > 3.0)
    clean_mask = (df_engineered['pacing_index'] > 0.5) & (df_engineered['pacing_index'] < 3.0)
    df_final = df_engineered[clean_mask].copy()
    
    print(f"Removed {len(df_engineered) - len(df_final)} extreme pacing outliers.")
    
    print(f"Saving engineered data to {output_path}...")
    df_final.to_parquet(output_path, engine='pyarrow')
    
    # 7. REPORTING
    # ---------------------------------------------------------
    print("\n" + "="*40)
    print("FEATURE ENGINEERING REPORT (v2)")
    print("="*40)
    print(f"Total Runners Analyzed: {len(df_final)}")
    
    print("\n--- Sample Data (First 5 Rows) ---")
    print(df_final[['gender', 'half', 'half_seconds', 'finish_time', 'pct_slowdown', 'hit_the_wall']].head())
    
    print("\n--- The Wall Statistics ---")
    wall_counts = df_final['hit_the_wall'].value_counts(normalize=True) * 100
    print(f"Rate of runners hitting the wall (>20% slowdown): {wall_counts.get(True, 0):.2f}%")
    
    return df_final

# --- EXECUTION BLOCK ---
input_file = "Dataset_Berlin_Cleaned_Analysis_Ready.parquet"
output_file = "Dataset_Berlin_Features_Engineered.parquet"

df_features = perform_feature_engineering_v2(input_file, output_file)

--- Starting Feature Engineering (v2) on Dataset_Berlin_Cleaned_Analysis_Ready.parquet ---
Data Loaded successfully. Shape: (873334, 18)
Diagnosing 'half' column...
Original Dtype: category
Sample values: ['01:03:54', '01:03:54', '01:03:54']
Parsing 'half' split to seconds...
Filtered 664 rows (0.08%) due to missing/invalid split data.
Calculating Pacing Index and Slowdown metrics...
Assigning Performance Categories...
Removed 48 extreme pacing outliers.
Saving engineered data to Dataset_Berlin_Features_Engineered.parquet...

FEATURE ENGINEERING REPORT (v2)
Total Runners Analyzed: 872622

--- Sample Data (First 5 Rows) ---
  gender      half  half_seconds     finish_time  pct_slowdown  hit_the_wall
0      M  01:03:54        3834.0 0 days 02:06:44     -1.669275         False
1      M  01:03:54        3834.0 0 days 02:06:57     -1.330203         False
2      M  01:03:54        3834.0 0 days 02:08:31      1.121544         False
3      M  01:03:55        3835.0 0 days 02:09:56      3.28552

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import skew, kurtosis
import numpy as np
import warnings

# Suppress warnings to keep the report clean
warnings.filterwarnings('ignore')

def generate_comprehensive_analysis(input_path):
    print(f"--- Loading Data from {input_path} ---")
    try:
        df = pd.read_parquet(input_path)
    except Exception as e:
        print(f"Error loading parquet: {e}")
        return

    # =========================================================
    # PART 1: TEXTUAL REPORT & STATISTICAL INSIGHTS (MACRO VIEW)
    # =========================================================
    # Filter for statistical stability in text report (removing extreme outliers just for the calc)
    df_clean = df[(df['pct_slowdown'] > -20) & (df['pct_slowdown'] < 100)].copy()
    
    print("\n" + "="*60)
    print("MACRO VIEW: INTELLIGENCE REPORT (GENDER PACING DYNAMICS)")
    print("="*60)

    # 1. Descriptive Statistics
    stats_df = df_clean.groupby('gender')['pct_slowdown'].agg(
        Mean='mean',
        Median='median',
        Std_Dev='std',
        Count='count'
    ).round(2)
    
    # Add Skewness (measure of the "Long Tail" of crashing)
    stats_df['Skewness'] = df_clean.groupby('gender')['pct_slowdown'].apply(skew).round(2)
    
    print("\n[1] KEY METRICS TABLE:")
    print(stats_df)
    
    # 2. Strategy Segmentation
    total_m = stats_df.loc['M', 'Count']
    total_f = stats_df.loc['F', 'Count']
    
    wall_m = len(df_clean[(df_clean['gender'] == 'M') & (df_clean['hit_the_wall'] == True)])
    wall_f = len(df_clean[(df_clean['gender'] == 'F') & (df_clean['hit_the_wall'] == True)])
    
    print(f"\n[2] THE WALL ANALYSIS (>20% Slowdown):")
    print(f"   - Men hitting the wall:   {wall_m:,} ({wall_m/total_m:.2%})")
    print(f"   - Women hitting the wall: {wall_f:,} ({wall_f/total_f:.2%})")
    
    # 3. Insight Generation
    diff_wall = (wall_m/total_m) - (wall_f/total_f)
    print(f"\n[3] EXECUTIVE INSIGHT:")
    print(f"   Men are {diff_wall*100:.1f} percentage points more likely to hit the wall than women.")
    if stats_df.loc['M', 'Skewness'] > stats_df.loc['F', 'Skewness']:
        print("   The higher skewness in men confirms a behavioral tendency towards extreme positive splits.")
    
    print("="*60 + "\n")

   # =========================================================
    # PART 2: PUBLICATION-READY PLOTTING 
    # =========================================================
    
    # 1. Global Aesthetics (Clean & Professional)
    sns.set_theme(style="white", font="sans-serif", font_scale=1.2)
    plt.rcParams.update({
        'font.family': 'sans-serif',
        'font.sans-serif': ['Arial', 'Helvetica', 'DejaVu Sans'],
        'axes.spines.top': False,
        'axes.spines.right': False,
        'axes.linewidth': 1.5,
        'xtick.major.width': 1.5,
        'ytick.major.width': 1.5
    })
    
    # Standard Figure Size for Journal (approx 2 columns width)
    fig_size = (10, 8) 
    custom_palette = {'M': '#404040', 'F': '#A31F34'} # Grey vs Red

    # --- FIGURE 1: DENSITY DISTRIBUTION (KDE) ---
    print("Generating Figure 1 (Density Distribution - Publishable)...")
    fig1, ax1 = plt.subplots(figsize=fig_size, dpi=300)
    
    # Plot KDE
    sns.kdeplot(
        data=df, 
        x='pct_slowdown', 
        hue='gender', 
        fill=True, 
        common_norm=False, 
        palette=custom_palette, 
        alpha=0.4, 
        linewidth=2.5, 
        ax=ax1
    )
    
    # Labels & Titles (No Chart Title for Paper - Legends usually go in caption)
    ax1.set_xlabel('Pacing Strategy (% Slowdown in 2nd Half)', fontsize=14, fontweight='bold', labelpad=10)
    ax1.set_ylabel('Density of Runners', fontsize=14, fontweight='bold', labelpad=10)
    
    # Reference Lines (Thinner, cleaner)
    ax1.axvline(0, color='black', linestyle='--', linewidth=1.5, alpha=0.8)
    ax1.text(0.5, ax1.get_ylim()[1]*0.95, 'Even Split', ha='left', fontsize=10, color='black')
    
    ax1.axvline(20, color='gray', linestyle=':', linewidth=1.5, alpha=0.8)
    ax1.text(20.5, ax1.get_ylim()[1]*0.95, 'The Wall (>20%)', ha='left', fontsize=10, color='gray')
    
    ax1.set_xlim(-15, 60)
    
    # Clean Legend
    sns.move_legend(ax1, "upper right", frameon=False, title=None, fontsize=12)
    
    # Save
    out1 = 'Figure_1_Density.tiff'
    print(f"Saving {out1}...")
    fig1.savefig(out1, dpi=300, format='tiff', pil_kwargs={"compression": "tiff_lzw"}, bbox_inches='tight')
    plt.close(fig1)

    # --- FIGURE 2: BOXPLOT WITH STATISTICS ---
    print("Generating Figure 2 (Boxplot with Stats - Publishable)...")
    fig2, ax2 = plt.subplots(figsize=(12, 8), dpi=300) # Slightly wider for categories
    
    order_cat = [
        '1. Elite/Sub-3h (<3:00)',
        '2. Advanced (3:00-3:30)',
        '3. Intermediate (3:30-4:00)',
        '4. Recreational (4:00-4:30)',
        '5. Casual (>4:30)'
    ]
    
    # Boxplot
    boxplot = sns.boxplot(
        data=df, 
        x='performance_level', 
        y='pct_slowdown', 
        hue='gender', 
        order=order_cat, 
        palette=custom_palette, 
        showfliers=False, 
        linewidth=1.5, 
        width=0.7,
        ax=ax2
    )
    
    # Add transparency manually to boxes
    for patch in boxplot.patches:
        r, g, b, a = patch.get_facecolor()
        patch.set_facecolor((r, g, b, 0.6))
    
    ax2.set_xlabel('Performance Category', fontsize=14, fontweight='bold', labelpad=10)
    ax2.set_ylabel('% Slowdown (2nd Half)', fontsize=14, fontweight='bold', labelpad=10)
    
    # Clean X-axis labels
    clean_labels = [l.split('(')[0].replace('1. ', '').replace('2. ', '').replace('3. ', '').replace('4. ', '').replace('5. ', '').strip() for l in order_cat]
    ax2.set_xticklabels(clean_labels, fontsize=11, rotation=0)
    
    # Reference Lines
    ax2.axhline(0, color='black', linestyle='--', linewidth=1, alpha=0.5)
    ax2.axhline(20, color='gray', linestyle=':', linewidth=1, alpha=0.5)
    
    # Statistical Annotation Loop (Recalibrated for new size)
    print("Calculating Statistical Significance...")
    y_max_plot = 0
    
    for i, category in enumerate(order_cat):
        cat_data = df[df['performance_level'] == category]
        men = cat_data[cat_data['gender'] == 'M']['pct_slowdown']
        women = cat_data[cat_data['gender'] == 'F']['pct_slowdown']
        
        if len(men) > 10 and len(women) > 10:
            stat, p_val = stats.mannwhitneyu(men, women, alternative='greater')
            
            # Cohen's d
            n1, n2 = len(men), len(women)
            s1, s2 = men.std(), women.std()
            m1, m2 = men.mean(), women.mean()
            pooled_std = np.sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))
            cohens_d = (m1 - m2) / pooled_std
            
            # Determine annotation
            if p_val < 0.001:
                sig_text = "***"
                font_w = 'bold'
            elif p_val < 0.05:
                sig_text = "*"
                font_w = 'bold'
            else:
                sig_text = "ns"
                font_w = 'normal'

            annot_label = f"{sig_text}\n(d={cohens_d:.2f})"
            
            # Coordinates
            x1, x2 = i - 0.2, i + 0.2
            y_box_top = cat_data['pct_slowdown'].quantile(0.85) # Use 85th percentile to avoid outliers
            y_line = y_box_top + 3
            h_line = 1
            
            # Draw
            col = 'black' if p_val < 0.05 else 'gray'
            ax2.plot([x1, x1, x2, x2], [y_line, y_line+h_line, y_line+h_line, y_line], lw=1.2, c=col)
            ax2.text((x1+x2)*.5, y_line + h_line + 0.5, annot_label, ha='center', va='bottom', 
                     color=col, fontsize=9, fontweight=font_w)
            
            if y_line > y_max_plot: y_max_plot = y_line

    ax2.set_ylim(bottom=-20, top=y_max_plot + 25) # Dynamic Y limit
    sns.move_legend(ax2, "upper left", frameon=False, title=None, fontsize=12)

    out2 = 'Figure_2_Boxplot.tiff'
    print(f"Saving {out2}...")
    fig2.savefig(out2, dpi=300, format='tiff', pil_kwargs={"compression": "tiff_lzw"}, bbox_inches='tight')
    plt.close(fig2)

    print("\n[SUCCESS] Report printed and Images generated.")

# --- EXECUTION ---
generate_comprehensive_analysis("Dataset_Berlin_Features_Engineered.parquet")

--- Loading Data from Dataset_Berlin_Features_Engineered.parquet ---

MACRO VIEW: INTELLIGENCE REPORT (GENDER PACING DYNAMICS)

[1] KEY METRICS TABLE:
         Mean  Median  Std_Dev   Count  Skewness
gender                                          
F        8.34    6.71     8.85  213826      1.33
M       10.71    8.09    11.24  658506      1.41

[2] THE WALL ANALYSIS (>20% Slowdown):
   - Men hitting the wall:   115,995 (17.61%)
   - Women hitting the wall: 20,657 (9.66%)

[3] EXECUTIVE INSIGHT:
   Men are 8.0 percentage points more likely to hit the wall than women.
   The higher skewness in men confirms a behavioral tendency towards extreme positive splits.

Generating Figure 1 (Density Distribution - Publishable)...
Saving Figure_1_Density.tiff...
Generating Figure 2 (Boxplot with Stats - Publishable)...
Calculating Statistical Significance...
Saving Figure_2_Boxplot.tiff...

[SUCCESS] Report printed and Images generated.


In [15]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

def perform_statistical_inference_report(input_path):
    """
    PERFORMS: DEEP STATISTICAL INFERENCE & HYPOTHESIS TESTING
    
    Tests Covered:
    1. Levene's Test (Assumption Check for Variance)
    2. Welch's T-Test (Difference of Means - Robust to unequal variance)
    3. Mann-Whitney U (Non-Parametric distribution check)
    4. Cohen's d (Effect Size)
    5. Chi-Square & Odds Ratio (Dependency of 'Hitting the Wall' on Gender)
    """
    
    print(f"--- Loading Data for Statistical Inference from {input_path} ---")
    try:
        df = pd.read_parquet(input_path)
    except Exception as e:
        print(f"Error loading parquet: {e}")
        return

    # Filter for analysis stability (-20% to +100% slowdown range)
    df_clean = df[(df['pct_slowdown'] > -20) & (df['pct_slowdown'] < 100)].copy()
    
    men = df_clean[df_clean['gender'] == 'M']['pct_slowdown']
    women = df_clean[df_clean['gender'] == 'F']['pct_slowdown']
    
    n_men = len(men)
    n_women = len(women)

    print("\n" + "="*80)
    print("STATISTICAL INFERENCE REPORT: GENDER DIFFERENCES IN MARATHON PACING")
    print("="*80)

    # ---------------------------------------------------------
    # 1. ASSUMPTION CHECKING
    # ---------------------------------------------------------
    print("\n[1] CHECKING ASSUMPTIONS")
    print("-" * 30)
    print(f"Sample Sizes: Men (N={n_men:,}), Women (N={n_women:,})")
    print("   -> STATUS: Large sample size (Central Limit Theorem applies).")
    print("      Normality is NOT required for T-test validity here.")
    
    # Levene's Test (Homogeneity of Variance)
    stat_levene, p_levene = stats.levene(men, women)
    print(f"\nLevene's Test (Equality of Variances):")
    print(f"   Statistic={stat_levene:.2f}, p-value={p_levene:.5e}")
    if p_levene < 0.05:
        print("   -> RESULT: Variances are UNEQUAL (Heteroscedasticity).")
        print("   -> DECISION: Using Welch's T-test (uncorrected for variance).")
    else:
        print("   -> RESULT: Variances are Equal.")

    # ---------------------------------------------------------
    # 2. HYPOTHESIS TESTING (PACING STRATEGY)
    # ---------------------------------------------------------
    print("\n" + "="*80)
    print("[2] PACING STRATEGY COMPARISON (CONTINUOUS VARIABLE)")
    print("-" * 30)

    # A. Welch's T-Test
    # H0: Means are equal
    t_stat, p_t = stats.ttest_ind(men, women, equal_var=False) 
    
    print("\nA. Welch's T-Test (Difference of Means):")
    print(f"   Mean Slowdown: Men = {men.mean():.2f}% | Women = {women.mean():.2f}%")
    print(f"   Difference (Delta): {men.mean() - women.mean():.2f} percentage points")
    print(f"   p-value: {p_t:.5e}")
    if p_t < 0.001: print("   -> CONCLUSION: Statistically Significant Difference (p < 0.001).")
    
    # B. Mann-Whitney U Test
    # H0: Distributions are equal
    u_stat, p_u = stats.mannwhitneyu(men, women, alternative='greater')
    
    print("\nB. Mann-Whitney U Test (Distribution Rank):")
    print(f"   p-value: {p_u:.5e}")
    if p_u < 0.001: print("   -> CONCLUSION: Men's pacing distribution is stochastically 'slower' (worse) than Women's.")

    # C. Cohen's d (Effect Size)
    pooled_std = np.sqrt(((n_men - 1) * men.std()**2 + (n_women - 1) * women.std()**2) / (n_men + n_women - 2))
    cohens_d = (men.mean() - women.mean()) / pooled_std
    
    print("\nC. Cohen's d (Effect Size):")
    print(f"   d = {cohens_d:.4f}")
    
    effect_label = "Negligible"
    if abs(cohens_d) >= 0.2: effect_label = "Small"
    if abs(cohens_d) >= 0.5: effect_label = "Medium"
    if abs(cohens_d) >= 0.8: effect_label = "Large"
    print(f"   -> INTERPRETATION: The biological difference is '{effect_label}'.")

    # ---------------------------------------------------------
    # 3. RISK ANALYSIS (HITTING THE WALL)
    # ---------------------------------------------------------
    print("\n" + "="*80)
    print("[3] RISK ANALYSIS: PROBABILITY OF 'HITTING THE WALL' (>20% SLOWDOWN)")
    print("-" * 30)

    contingency = pd.crosstab(df_clean['gender'], df_clean['hit_the_wall'])
    
    # A. Chi-Square Test
    chi2, p_chi2, dof, ex = stats.chi2_contingency(contingency)
    
    print(f"Chi-Square Test of Independence:")
    print(f"   Chi2 Stat: {chi2:.2f}, p-value: {p_chi2:.5e}")
    
    # B. Odds Ratio & Probabilities
    p_m = contingency.loc['M', True] / (contingency.loc['M', True] + contingency.loc['M', False])
    p_f = contingency.loc['F', True] / (contingency.loc['F', True] + contingency.loc['F', False])
    
    odds_m = p_m / (1 - p_m)
    odds_f = p_f / (1 - p_f)
    odds_ratio = odds_m / odds_f
    
    print(f"\nRisk Metrics:")
    print(f"   Probability (Men):   {p_m:.2%}")
    print(f"   Probability (Women): {p_f:.2%}")
    print(f"   Odds Ratio: {odds_ratio:.2f}")
    print(f"   -> INSIGHT: Men are {odds_ratio:.2f}x times more likely to hit the wall than women.")

    
    # ---------------------------------------------------------
    # 4. VISUALIZATION
    # ---------------------------------------------------------
    print("\n[VISUALIZATION] Generating Risk Plot ...")
    
    # --- CONFIGURATION: Reset to Standard Publication Sizes ---
    # Instead of poster size, we use standard journal dimensions (approx 8x8 inches)
    fig_size = (8, 8)
    
    # Reset rcParams to ensure fonts are readable on A4 paper (10-14pt range)
    plt.rcParams.update({
        'font.family': 'sans-serif',
        'font.sans-serif': ['Arial', 'Helvetica', 'DejaVu Sans'],
        'font.size': 12,              # General text
        'axes.labelsize': 14,         # Axis labels (Bold usually)
        'axes.titlesize': 14,         # Title (if used)
        'xtick.labelsize': 12,        # Tick labels
        'ytick.labelsize': 12,
        'lines.linewidth': 1.5,       # Thinner lines for paper
        'axes.linewidth': 1.5,        # Axis spine width
        'axes.spines.top': False,     # Remove top border
        'axes.spines.right': False    # Remove right border
    })
    
    fig, ax = plt.subplots(figsize=fig_size, dpi=300)
    
    # Data Preparation
    probs = [p_m*100, p_f*100]
    genders = ['Men', 'Women']
    colors = ['#404040', '#A31F34'] # Dark Grey vs Red
    
    # --- PLOTTING: Clean Bar Chart ---
    # Using alpha=0.9 for solid but not harsh color
    bars = ax.bar(genders, probs, color=colors, width=0.6, 
                  alpha=0.9, edgecolor='black', linewidth=1.5)
    
    # --- ERROR BARS (95% CI) ---
    # Calculate Standard Error
    se = [np.sqrt(p*(1-p)/n)*100 for p, n in zip([p_m, p_f], [n_men, n_women])]
    
    # Capsize reduced to 5 (standard for figures), thinner lines
    ax.errorbar(genders, probs, yerr=[1.96*s for s in se], 
                fmt='none', ecolor='black', capsize=5, elinewidth=1.5, markeredgewidth=1.5)
    
    # --- LABELS & ANNOTATIONS ---
    ax.set_ylabel('Probability of Hitting the Wall (%)', fontsize=14, fontweight='bold', labelpad=10)
    
    
    ax.set_title('') 
    
    # Bar Labels (Percentages)
    ax.bar_label(bars, fmt='%.1f%%', padding=4, fontsize=14, fontweight='bold')
    
    # --- ODDS RATIO ANNOTATION (Professional Box) ---
    # Placed centrally above the highest bar
    annot_text = f"Odds Ratio: {odds_ratio:.2f}\n(Twice the Risk)\np < 0.001"
    
    # Dynamic Y positioning
    y_pos_text = max(probs) * 1.25
    
    ax.text(0.5, y_pos_text, annot_text, 
            ha='center', va='center', fontsize=12, fontweight='bold', color='black',
            bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5', linewidth=1))
    
    # Adjust Y-Limit to make room for annotation
    ax.set_ylim(0, max(probs) * 1.5)
    
    # Light horizontal grid for readability
    ax.grid(axis='y', linestyle=':', alpha=0.3)
    
    # --- SAVING (TIFF Format preferred) ---
    out_file = 'Figure_3_Risk_Plot.tiff'
    print(f"Saving Risk Plot to {out_file}...")
    
    # Using PIL kwargs for compression (LZW is standard for TIFF)
    plt.savefig(out_file, dpi=300, format='tiff', pil_kwargs={"compression": "tiff_lzw"}, bbox_inches='tight')
    plt.close()
    
    print("Success. Figure 3 generated.")

# --- EXECUTION ---
# Use the file generated in Step 1 (Feature Engineering)
perform_statistical_inference_report("Dataset_Berlin_Features_Engineered.parquet")

--- Loading Data for Statistical Inference from Dataset_Berlin_Features_Engineered.parquet ---

STATISTICAL INFERENCE REPORT: GENDER DIFFERENCES IN MARATHON PACING

[1] CHECKING ASSUMPTIONS
------------------------------
Sample Sizes: Men (N=658,506), Women (N=213,826)
   -> STATUS: Large sample size (Central Limit Theorem applies).
      Normality is NOT required for T-test validity here.

Levene's Test (Equality of Variances):
   Statistic=8218.14, p-value=0.00000e+00
   -> RESULT: Variances are UNEQUAL (Heteroscedasticity).
   -> DECISION: Using Welch's T-test (uncorrected for variance).

[2] PACING STRATEGY COMPARISON (CONTINUOUS VARIABLE)
------------------------------

A. Welch's T-Test (Difference of Means):
   Mean Slowdown: Men = 10.71% | Women = 8.34%
   Difference (Delta): 2.37 percentage points
   p-value: 0.00000e+00
   -> CONCLUSION: Statistically Significant Difference (p < 0.001).

B. Mann-Whitney U Test (Distribution Rank):
   p-value: 0.00000e+00
   -> CONCLUSION: Men

In [28]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import warnings

# Suppress repetitive warnings for cleaner output
warnings.filterwarnings('ignore')

def stratified_micro_analysis_report(input_path):
    """
    PERFORMS: MICRO VIEW - STRATIFIED ANALYSIS
    
    Goals:
    1. Loop through each Performance Category (Sub-3h, Advanced, etc.)
    2. Perform Independent Statistical Tests (Mann-Whitney U) for each stratum.
    3. Calculate the 'Ego Gap' (Difference in Slowdown) for each level.
    4. Determine if the behavioral difference persists across all fitness levels.
    """
    
    print(f"--- Loading Data for Micro-Analysis from {input_path} ---")
    try:
        df = pd.read_parquet(input_path)
    except Exception as e:
        print(f"Error loading parquet: {e}")
        return

    # Filter for stability (remove extreme outliers for accurate stats)
    df_clean = df[(df['pct_slowdown'] > -20) & (df['pct_slowdown'] < 100)].copy()

    # Define Categories in Order of Performance
    categories = [
        '1. Elite/Sub-3h (<3:00)',
        '2. Advanced (3:00-3:30)',
        '3. Intermediate (3:30-4:00)',
        '4. Recreational (4:00-4:30)',
        '5. Casual (>4:30)'
    ]

    print("\n" + "="*80)
    print("MICRO VIEW: STRATIFIED ANALYSIS OF THE 'EGO CURSE'")
    print("Does the gender gap persist when controlling for performance level?")
    print("="*80)

    summary_data = []

    for cat in categories:
        print(f"\n>>> ANALYZING CATEGORY: {cat}")
        print("-" * 60)
        
        # Subset Data for this specific category
        subset = df_clean[df_clean['performance_level'] == cat]
        men = subset[subset['gender'] == 'M']['pct_slowdown']
        women = subset[subset['gender'] == 'F']['pct_slowdown']
        
        n_m, n_f = len(men), len(women)
        
        # Check if enough data exists
        if n_m < 50 or n_f < 50:
            print(f"   [WARNING] Insufficient data for this category (M={n_m}, F={n_f}). Skipping.")
            continue

        # 1. Descriptive Stats
        mean_m, mean_f = men.mean(), women.mean()
        delta = mean_m - mean_f
        
        # 2. Mann-Whitney U Test (Hypothesis Testing)
        # H0: Distribution of Men = Distribution of Women
        # H1: Men > Women (One-sided)
        u_stat, p_val = stats.mannwhitneyu(men, women, alternative='greater')
        
        # 3. Cohen's d (Effect Size)
        # pooled_std = sqrt( ((n1-1)s1^2 + (n2-1)s2^2) / (n1+n2-2) )
        pooled_std = np.sqrt(((n_m - 1) * men.std()**2 + (n_f - 1) * women.std()**2) / (n_m + n_f - 2))
        cohens_d = (mean_m - mean_f) / pooled_std
        
        # 4. Wall Hit Rate (Risk Analysis)
        wall_m = len(subset[(subset['gender'] == 'M') & (subset['hit_the_wall'] == True)])
        wall_f = len(subset[(subset['gender'] == 'F') & (subset['hit_the_wall'] == True)])
        
        rate_m = (wall_m / n_m) * 100
        rate_f = (wall_f / n_f) * 100
        risk_ratio = rate_m / rate_f if rate_f > 0 else 0

        # REPORTING THE FINDINGS
        print(f"   Sample: Men (N={n_m:,}), Women (N={n_f:,})")
        print(f"   Avg Slowdown: Men = {mean_m:.2f}%, Women = {mean_f:.2f}%")
        print(f"   The 'Ego Gap' (Delta): {delta:.2f} percentage points (Men are slower)")
        
        sig_label = "SIGNIFICANT" if p_val < 0.001 else "NOT Significant"
        print(f"   Statistical Significance (p-value): {p_val:.2e} -> {sig_label}")
        print(f"   Effect Size (Cohen's d): {cohens_d:.3f}")
        
        print(f"   Wall Hit Rate (>20% slowdown):")
        print(f"      Men: {rate_m:.2f}% | Women: {rate_f:.2f}%")
        print(f"      Risk Multiplier: Men are {risk_ratio:.2f}x more likely to crash.")
        
        # Store for final summary table
        summary_data.append({
            'Category': cat,
            'Delta (pp)': delta,
            'Cohen d': cohens_d,
            'Risk Ratio': risk_ratio
        })

    # FINAL VERDICT TABLE
    print("\n" + "="*80)
    print("FINAL VERDICT: DOES THE CURSE EXIST AT ALL LEVELS?")
    print("="*80)
    print(f"{'Category':<30} | {'Ego Gap (pp)':<12} | {'Effect Size':<12} | {'Risk Ratio':<10}")
    print("-" * 75)
    
    consistent_evidence = True
    for item in summary_data:
        print(f"{item['Category']:<30} | +{item['Delta (pp)']:<11.2f} | {item['Cohen d']:<12.3f} | {item['Risk Ratio']:.2f}x")
        
        # Check consistency (if effect size drops below 0.1, we consider it disappearing)
        if item['Cohen d'] < 0.1:
            consistent_evidence = False
            
    print("-" * 75)
    
    if consistent_evidence:
        print("\n>>> CONCLUSION: YES. The 'Ego Curse' is omnipresent.")
        print("    Men consistently manage pacing worse than women across ALL performance levels,")
        print("    from Elite Amateurs to Casual Runners. Fitness does NOT cure bad pacing behavior.")
    else:
        print("\n>>> CONCLUSION: MIXED. The effect varies by level.")

# --- EXECUTION ---
stratified_micro_analysis_report("Dataset_Berlin_Features_Engineered.parquet")

--- Loading Data for Micro-Analysis from Dataset_Berlin_Features_Engineered.parquet ---

MICRO VIEW: STRATIFIED ANALYSIS OF THE 'EGO CURSE'
Does the gender gap persist when controlling for performance level?

>>> ANALYZING CATEGORY: 1. Elite/Sub-3h (<3:00)
------------------------------------------------------------
   Sample: Men (N=40,196), Women (N=2,131)
   Avg Slowdown: Men = 3.81%, Women = 2.74%
   The 'Ego Gap' (Delta): 1.07 percentage points (Men are slower)
   Statistical Significance (p-value): 1.35e-17 -> SIGNIFICANT
   Effect Size (Cohen's d): 0.210
   Wall Hit Rate (>20% slowdown):
      Men: 1.42% | Women: 0.23%
      Risk Multiplier: Men are 6.06x more likely to crash.

>>> ANALYZING CATEGORY: 2. Advanced (3:00-3:30)
------------------------------------------------------------
   Sample: Men (N=113,818), Women (N=12,202)
   Avg Slowdown: Men = 6.02%, Women = 3.38%
   The 'Ego Gap' (Delta): 2.64 percentage points (Men are slower)
   Statistical Significance (p-value): 0.0

In [5]:
import pandas as pd

def generate_detailed_demographics(input_path):
    print(f"--- Generating Expanded Sample Details from {input_path} ---")
    df = pd.read_parquet(input_path)
    
    total_n = len(df)
    
    # 1. Gender Demographics
    counts = df['gender'].value_counts()
    pcts = df['gender'].value_counts(normalize=True) * 100
    
    print("\n[1] GENDER BREAKDOWN (Fill in the Text):")
    print(f"   > Men (N): {counts.get('M', 0):,} ({pcts.get('M', 0):.1f}%)")
    print(f"   > Women (N): {counts.get('F', 0):,} ({pcts.get('F', 0):.1f}%)")
    
    # 2. Age Distribution
    print("\n[2] TOP 3 AGE GROUPS:")
    age_counts = df['age_group'].value_counts().head(3)
    for age, count in age_counts.items():
        print(f"   > {age}: {count:,} runners ({count/total_n:.1f}%)")

    # 3. Advanced Time Stats (IQR)
    print("\n[3] FINISH TIME ROBUSTNESS (Median & IQR):")
    # Helper for HH:MM:SS
    def fmt(seconds):
        h = int(seconds // 3600); m = int((seconds % 3600) // 60); s = int(seconds % 60)
        return f"{h}:{m:02d}:{s:02d}"

    stats = df.groupby('gender')['time_seconds'].describe(percentiles=[0.25, 0.75])
    
    for g in ['M', 'F']:
        median = fmt(stats.loc[g, '50%'])
        p25 = fmt(stats.loc[g, '25%'])
        p75 = fmt(stats.loc[g, '75%'])
        print(f"   > {g} Median: {median} | IQR: {p25} – {p75}")

generate_detailed_demographics("Dataset_Berlin_Cleaned_Analysis_Ready.parquet")

--- Generating Expanded Sample Details from Dataset_Berlin_Cleaned_Analysis_Ready.parquet ---

[1] GENDER BREAKDOWN (Fill in the Text):
   > Men (N): 659,294 (75.5%)
   > Women (N): 214,040 (24.5%)

[2] TOP 3 AGE GROUPS:
   > 40: 162,234 runners (0.2%)
   > 35: 143,728 runners (0.2%)
   > 45: 142,610 runners (0.2%)

[3] FINISH TIME ROBUSTNESS (Median & IQR):
   > M Median: 3:57:46 | IQR: 3:32:02 – 4:28:49
   > F Median: 4:26:02 | IQR: 3:58:20 – 4:55:40
